In [1]:
import sys
sys.path.append('..')
wp = '/home/jose/lung_cancer_ds_bowl/'

In [ ]:
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from utils import plotting
from dl_model_pos_patches import  common
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.sample_resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from dl_model_patches import  common
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.sample_resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard



# PATHS
wp = os.environ['LUNG_PATH']


INPUT_PATH = '/mnt/hd2/preprocessed5'  # INPUT_PATH = wp + 'data/preprocessed5_sample'
VALIDATION_PATH = '/mnt/hd2/preprocessed5_validation_luna'
NODULES_PATH = wp + 'data/luna/annotations.csv'
#PATCHES_PATH = '/mnt/hd2/patches'  # PATCHES_PATH = wp + 'data/preprocessed5_patches'
PATCHES_PATH = '/home/jose/patches_temp'

OUTPUT_MODEL = wp + 'models/jm_patches_train_v17.hdf5'  # OUTPUT_MODEL = wp + 'personal/jm_patches_train_v06_local.hdf5'
LOGS_PATH = wp + 'logs/%s' % str('v17')



## TRAINING -----------------------------------------------------------------


def chunks_multichannel(X, y, batch_size=32, augmentation_times=4, thickness=0, is_training=True):
    """
    Batches generator for keras fit_generator. Returns batches of patches 40x40px
     - augmentation_times: number of time to return the data augmented
     - concurrent_patients: number of patients to load at the same time to add diversity
     - thickness: number of slices up and down to add as a channel to the patch
    """
    y = np.expand_dims(y, axis=1)
    while 1:
        # downsample negatives (reduce 90%)
        if is_training:
            len1 = int(0.25*batch_size)
            idx_1 = [i for i in range(len(y)) if y[i]==1]
            idx_1 = random.sample(idx_1, len1)
            idx_0 = [i for i in range(len(y)) if y[i]==0]
            idx_0 = random.sample(idx_0, batch_size - len1)
            selected_samples = idx_0 + idx_1
            random.shuffle(selected_samples)
        else:
            selected_samples = range(len(y))

        #selected_samples  = [i for i in range(len(y_orig)) if y_orig[i]==1 or random.randint(0,9)==0]
        X = X[selected_samples]
        y = y[selected_samples]
        logging.info("Final downsampled dataset stats: TP:%d, FP:%d" % (sum(y), len(y)-sum(y)))


        a = np.array([X[i][0] for i in range(len(X))])
        b = np.array([X[i][1][2:18] for i in range(len(X))])
        yield [a,b], y

# LOADING PATCHES FROM DISK
logging.info("Loading training and test sets")
x_train = np.load(os.path.join(PATCHES_PATH, 'x_train_dl_pos_0.npz'))['arr_0']
y_train = np.load(os.path.join(PATCHES_PATH, 'y_train_dl_pos_0.npz'))['arr_0']
x_test = np.load(os.path.join(PATCHES_PATH, 'x_test_dl_pos_0.npz'))['arr_0']
y_test = np.load(os.path.join(PATCHES_PATH, 'y_test_dl_pos_0.npz'))['arr_0']
logging.info("Training set (1s/total): %d/%d" % (sum(y_train),len(y_train)))
logging.info("Test set (1s/total): %d/%d" % (sum(y_test), len(y_test)))


from dl_networks.pos_resnet import posResnet
model = posResnet().get_posResnet((3,40,40),(16,18,18))
# Load model
#model = ResnetBuilder().build_resnet_34((5,40,40),1)
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy','fmeasure'])
# logging.info('Loading exiting model...')
# model.load_weights(OUTPUT_MODEL)


model.fit_generator(
                    generator=chunks_multichannel(x_train, y_train, batch_size=32, thickness=2),
                    samples_per_epoch=1280,  # make it small to update TB and CHECKPOINT frequently
                    nb_epoch=500,
                    verbose=1,
                    #class_weight={0:1., 1:4.},
                    #callbacks=[tb, model_checkpoint],
                    validation_data=chunks_multichannel(x_test, y_test, batch_size=32, thickness=2),  # TODO: is_training=False
                    nb_val_samples=32*10,
                    max_q_size=64,
                    nb_worker=1)  # a locker is needed if increased the number of parallel workers



04-02 16:38:25  INFO     Loading training and test sets
04-02 16:38:29  INFO     Training set (1s/total): 31/817
04-02 16:38:29  INFO     Test set (1s/total): 79/2128
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24


Epoch 1/500


04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO     Final downsampled dataset stats: TP:8, FP:24
04-02 16:38:47  INFO

  32/1280 [..............................] - ETA: 333s - loss: 3.3654 - acc: 0.5312 - fmeasure: 0.4000

04-02 16:38:55  INFO     Final downsampled dataset stats: TP:8, FP:24


...

04-02 16:39:08  INFO     Final downsampled dataset stats: TP:8, FP:24


 192/1280 [===>..........................] - ETA: 140s - loss: 2.9919 - acc: 0.8542 - fmeasure: 0.7346

04-02 16:39:12  INFO     Final downsampled dataset stats: TP:8, FP:24


 224/1280 [====>.........................] - ETA: 132s - loss: 2.9659 - acc: 0.8661 - fmeasure: 0.7521

04-02 16:39:15  INFO     Final downsampled dataset stats: TP:8, FP:24
